In [ ]:
pip install tensorflow scikit-learn pandas matplotlib numpy

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

dataset_path = "Food-5K/training"  # Replace with your actual training folder path
validation_path = "Food-5K/validation"  # Replace with your actual validation folder path

# Image generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# CNN model for food image classification
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Change to the number of categories in your dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the food and calories dataset
calorie_df = pd.read_csv("Food and Calories.csv")  # Replace with your file path

# Preprocess the dataset (cleaning food names)
calorie_df['Food'] = calorie_df['Food'].apply(lambda x: x.lower().strip())

# Label encode the food names for matching
food_names = calorie_df['Food'].unique()
label_encoder = LabelEncoder()
label_encoder.fit(food_names)
calorie_df['Food_encoded'] = label_encoder.transform(calorie_df['Food'])

# Function to get calorie estimate based on food name
def get_calories(food_name):
    food_name = food_name.lower().strip()
    try:
        food_label = label_encoder.transform([food_name])[0]
        food_data = calorie_df[calorie_df['Food_encoded'] == food_label]
        return food_data['Calories'].values[0] if not food_data.empty else None
    except:
        return None

In [ ]:
# Function to classify food and estimate calories
def classify_and_get_calories(image_path):
    # Load and preprocess the image (resize and normalize)
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Normalize image

    # Add batch dimension for prediction (the model expects a batch of images)
    img_array = tf.expand_dims(img_array, 0)

    # Predict the class
    predicted_class = model.predict(img_array)
    predicted_class_index = tf.argmax(predicted_class, axis=1).numpy()[0]

    # Get the food name corresponding to the predicted class
    predicted_food_name = label_encoder.inverse_transform([predicted_class_index])[0]

    # Get the calorie estimate for the predicted food
    calories = get_calories(predicted_food_name)

    return predicted_food_name, calories

# Example usage: classify an image and estimate calories
image_path = "Pizza.jpg"  # Replace with your image path
predicted_food, predicted_calories = classify_and_get_calories(image_path)

print(f"The predicted food is: {predicted_food}, with {predicted_calories} calories.")